In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob

from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

In [ ]:
basedir = '/data/u934/service_imagerie/v_kapoor/StarDistData/TimeData/'
basedirResults = '/data/u934/service_imagerie/v_kapoor/StarDistData/JuliaResultsData/WithoutShapeCompletion/'
ModelName = 'UNet_model'

BaseDirModels = 'models'

In [ ]:
Path = os.path.join(basedir, '*.tif')
X = []
Y = []
Names = []

filesRaw = glob.glob(Path)
maxtime = 0
for fname in filesRaw:
     x = imread(fname)
     X.append(x)
     Names.append(fname)
     if X[0].shape[0] > maxtime:
      maxtime =  X[0].shape[0]  
X.sort
Names.sort
SegmentedLabels = [];


for i in range(len(X)):
    y = np.zeros((maxtime, X[0].shape[1], X[0].shape[2]))
    image = X[i]
    
    y[:X[i].shape[0],:,:] = image[:,:,:]
    Y.append(y) 
Y.sort
axes = 'YX'
print('Total number of images to be segmented = ', len(X))
print('Image size = ', X[0].shape)
print('Time dim = ', maxtime )
timerange = maxtime 
count = min(0, len(X))

In [ ]:
model = CARE(config = None, name = ModelName, basedir = BaseDir)

In [ ]:
def ApplyModelovertime(model,i):
      
    img = Y[i]
    for j in range(timerange):
      maxpoints = 0;
      secdim = 0;
      print('Time point = ' , j, 'Image number = ', i)
      smallimg = img[j,:,:]
      segmented = model.predict(x, axes);
      SegmentedLabels.append(segmented)
        
        
print(len(SegmentedLabels))
plt.figure(figsize=(13,10))
plt.subplot(121); plt.imshow(smallimg,cmap='gray'); plt.axis('off')
plt.subplot(121); plt.imshow(segmented,cmap='gray'); plt.axis('off')

In [ ]:
for i in range(0, len(Y)):
    
       ApplyModelovertime(ModelName ,i)

In [ ]:
(basedirResults)
print(len(SegmentedLabels))
for i in range(0, len(SegmentedLabels)):
           print('Saving file'  + basedirResults  + os.path.basename(Names[i]))
           save_tiff_imagej_compatible((basedirResults + os.path.basename(Names[i])) , SegmentedLabels[i], axes)   
           print(SegmentedLabels[i].shape,len(SegmentedLabels) )  

In [ ]:
plt.figure(figsize = (16, 10))
smallimg = img[timerange - 1,:,:]
Lastimage= SegmentedLabels[len(SegmentedLabels) - 1]
smallSegmentedLabels = Lastimage[timerange - 1,:,:]
plt.subplot(111); plt.imshow(smallimg, cmap = 'gray'); plt.axis('off'); plt.title('Raw Image')
plt.subplot(121); plt.imshow(smallSegmentedLabels , cmap = 'gray'); plt.axis('off'); plt.title('Segmentation (last timepoint of last image)')
